In [1]:
import datetime
import json
import time
import os
import datetime
import json
import time
from openai import AzureOpenAI
from dotenv import load_dotenv
import json
import copy
import textwrap

# cost per token for GPT4 8K
prompt_token_cost = 0.03/1000
generation_token_cost = 0.06/1000

In [2]:

# Load environment variables
load_dotenv()


True

Creating system prompt and user promot

In [3]:
model = os.getenv("MODEL")

user_messages = [
    "GPT-3 (Generative Pre-trained Transformer 3) - OpenAI's powerful language model capable of writing like a human",  
    "DALL-E - OpenAI's AI system that can create images from textual descriptions",  
    "Scientific research has led to significant advancements in medicine and healthcare.",  
    "CLIP (Contrastive Language-Image Pretraining) - OpenAI's model that understands images in the context of natural language",  
    "Science has contributed to our understanding of the natural world and the universe.",  
    "Codex - OpenAI's AI system that can understand and generate code, powering GitHub Copilot",  
    "GPT-4 - OpenAI's rumored next iteration of their language model with anticipated improvements",  
    "Azure AI - Microsoft's suite of AI services, including machine learning, cognitive services, and conversational AI",  
     "The collaboration and exchange of scientific knowledge across international borders have facilitated global progress in various fields." ,
     "Scientific innovations have improved communication and connectivity through technology.", 
    "Microsoft Turing Models - A series of large-scale language models developed by Microsoft",  
    "Microsoft Project Brainwave - Real-time AI platform for cloud and edge computing",  
    "Microsoft AI for Earth - A program applying AI to environmental challenges",  
    "Microsoft AI for Health - An initiative leveraging AI for health-related research",  
    "Scientific innovations have improved communication and connectivity through technology.",  
    "OpenAI's API - Providing access to GPT-3 and other models for various applications",   
]

system_message = '''
You are an helpful AI assistant that categorizes the text in one of the two categories : SCIENCE, AI. 
###Important :
Do not add any additional information.
Make sure to complete all elements of the array'''


Sequential Execution

In [8]:
client = AzureOpenAI(
        api_version=os.getenv("API_VERSION"),
        azure_endpoint=os.getenv("AZURE_ENDPOINT"),
        api_key=os.getenv("API_KEY")
    )

responses = []

start_time = time.time()

for i, user_message in enumerate(user_messages):

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message},
        ],
    )

    result = completion.choices[0].message.content
    responses.append(result)

end_time = time.time()
e2e_time = end_time - start_time

print(responses)
print("Time taken for execution : ", e2e_time)


AuthenticationError: Error code: 401 - {'statusCode': 401, 'message': 'Unauthorized. Access token is missing, invalid, audience is incorrect (https://cognitiveservices.azure.com), or have expired.'}

Batching Requests

In [4]:
def aoai_call(system_message,prompt, model):
    client = AzureOpenAI(
        api_version=os.getenv("API_VERSION"),
        azure_endpoint=os.getenv("AZURE_ENDPOINT"),
        api_key=os.getenv("API_KEY")
    )

    start_time = time.time()

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
    )

    end_time = time.time()
    e2e_time = end_time - start_time

    result=json.loads(completion.model_dump_json(indent=2))
    completion_text=result["choices"][0]["message"]["content"]

    return completion_text, e2e_time

aoai_call(system_message, 
          json.dumps([user_messages]), model)


('[\n  ["AI", "AI", "SCIENCE", "AI", "SCIENCE", "AI", "AI", "AI", "SCIENCE", "SCIENCE", "AI", "AI", "AI", "AI", "SCIENCE", "AI"]\n]',
 4.357616662979126)

Asynchronous call

In [5]:
import asyncio
import aiohttp
import os
from aiohttp import ClientSession

async def fetch(session, system_message, user_message):
    url = f'{os.getenv("AZURE_ENDPOINT")}/openai/deployments/gpt-4/chat/completions?api-version={os.getenv("API_VERSION")}'
    headers = {
        "Content-Type": "application/json",
        "api-key": os.getenv("API_KEY")
    }  
    data = {
        "model": "gpt-4",  # Adjust the model as needed
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ]
    }

    async with session.post(url, json=data, headers=headers) as response:
        return await response.json()

async def main(system_message, user_messages):
    api_call_batch_size = 16

    async with ClientSession() as session:
        tasks = []
        responses = []
        for i, user_message in enumerate(user_messages):
            task = asyncio.create_task(fetch(session, system_message, user_message))
            tasks.append(task)
            if len(tasks) >= api_call_batch_size: 
                responses.extend(await asyncio.gather(*tasks))
                tasks = []
        responses.extend(await asyncio.gather(*tasks))  # Process the last batch
        return responses

system_message = '''
You are an helpful AI assistant that categorizes the text in one of the two categories : SCIENCE, AI. 
###Important :
Do not add any additional information'''

start = time.time()
responses_async = await main(system_message, user_messages)
end = time.time()

run_time = end - start
print(f"Total time taken: {run_time} seconds")
# responses
response_content = []
for i in range(len(responses_async)):
    try:
        response_content.append(responses_async[i]["choices"][0]["message"]["content"])
    except:
        pass
print(response_content)
print(len(response_content))

Total time taken: 1.467106819152832 seconds
['AI', 'AI', 'SCIENCE', 'AI', 'SCIENCE', 'AI', 'AI', 'AI', 'SCIENCE', 'SCIENCE', 'AI', 'AI', 'AI', 'AI', 'SCIENCE', 'AI']
16


In [6]:
print(len(user_messages))

16


In [ ]:
s